In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import Packages.CoNLLReader as reader
import numpy as np
from tqdm import tqdm
from Levenshtein import distance
from collections import Counter

In [23]:
text, data = reader.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,tags,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",,,,,,
1,0,rejects,"(3, 10)",,,,,,
2,0,German,"(11, 17)",B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",,,,,,
4,0,to,"(23, 25)",,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1390, 1397)",,,,,,
285585,1392,brother,"(1398, 1405)",,,,,,
285586,1392,",","(1406, 1407)",,,,,,
285587,1392,Bobby,"(1408, 1413)",B,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


### General Info

In [24]:
n_entities = sum([x is not '' for x in list(data['entities'])])
n_ass_ents = sum([x is not '' for x in list(data['numeric_codes'])])
n_tokens = sum([1 for x in list(data['entities'])])

In [25]:
print('{0:<35} {1:>10} '.format("Numero totale di entità:", n_entities))
print('{0:<35} {1:>10} '.format("Numero totale di tokens:", n_tokens))
print('{0:<35} {1:>10} '.format("1 entità ogni:", round(n_tokens / n_entities, 2)))

Numero totale di entità:                 27817 
Numero totale di tokens:                285589 
1 entità ogni:                           10.27 


In [26]:
ents_data = data[data['entities'] != '']
golden_standard_entities = ents_data['entities'].values

### Gold standard

In [27]:
unique_entities = np.unique(golden_standard_entities)
golden_standard_dict = Counter(golden_standard_entities)
golden_standard_dict = dict(sorted(golden_standard_dict.items(), key=lambda item: item[1], reverse=True))

In [28]:
mentions = ents_data['mentions'].values
mentions = [x.lower() for x in mentions]

### Clustering by Levenshtein distance and DBSCAN

In [32]:
# Way with dbscan algorithm
import numpy as np
from sklearn.cluster import dbscan

clustering = False
if clustering:
    def lev_metric(x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        if len(mentions[i]) < 4:
            if mentions[i] == mentions[j]:
                return 0
            else:
                return distance(mentions[i].lower(), mentions[j].lower()) + 3
        else:
            return distance(mentions[i].lower(), mentions[j].lower())


    X = np.arange(len(mentions)).reshape(-1, 1)
    _, leven_cluster = dbscan(X, metric=lev_metric, eps=1, min_samples=1, n_jobs=-1)
    np.savetxt('db_cluster_levestein.txt', leven_cluster, delimiter=',')
else:
    leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0.txt", dtype=np.int32)

In [33]:
# Now we create a dict for each cluster that contains entities and entities count
lev_cluster_dict = {}
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append((mentions[i],golden_standard_entities[i]))
    except:
        lev_cluster_dict[x] = [(mentions[i],golden_standard_entities[i])]
# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [34]:
for key in tqdm(lev_cluster_dict):
    cluster_list = [x[1] for x in lev_cluster_dict[key]]
    cluster_dict = Counter(cluster_list)
    lev_cluster_dict[key] = cluster_dict
lev_cluster_dict

100%|██████████| 6066/6066 [00:00<00:00, 252477.36it/s]


{0: Counter({'Germany': 354,
          'Germany_Davis_Cup_team': 1,
          'German_language': 3,
          'Germany_national_football_team': 6,
          'Germans': 1,
          "Germany_men's_national_ice_hockey_team": 1,
          'Germany_Fed_Cup_team': 1,
          'Nazi_Germany': 2,
          'Lee_Germon': 2}),
 1: Counter({'United_Kingdom': 92, 'England': 1}),
 2: Counter({'Brussels': 44}),
 3: Counter({'European_Commission': 8}),
 4: Counter({'European_Union': 24}),
 5: Counter({'United_Kingdom': 166}),
 6: Counter({'European_Commission': 11}),
 7: Counter({'Franz_Fischler': 2}),
 8: Counter({'France': 193,
          'France_Fed_Cup_team': 1,
          'France_national_football_team': 8}),
 9: Counter({'Bovine_spongiform_encephalopathy': 11,
          'Budapest_Stock_Exchange': 2}),
 10: Counter({'Spain': 31, 'Spanish_language': 1}),
 11: Counter({'Loyola_de_Palacio': 1}),
 12: Counter({'Europe': 44}),
 13: Counter({'Bonn': 42}),
 14: Counter({'Jimi_Hendrix': 4}),
 15: Counte

### evaluation levestein

CEAFm precision

In [35]:
max_lev_cluster_list = []
for key in lev_cluster_dict.keys():
    max_key = max(lev_cluster_dict[key], key=lev_cluster_dict[key].get)
    max_lev_cluster_list.append((max_key, lev_cluster_dict[key][max_key]))
max_lev_cluster_list.sort(key=lambda x: x[1], reverse=True)
max_lev_cluster_dict = dict.fromkeys(golden_standard_entities, 0)
for key, val in max_lev_cluster_list:
    if max_lev_cluster_dict[key] == 0:
        max_lev_cluster_dict[key] = val

In [36]:
# CEAFm_levenshtein_precision
CEAFm_levenshtein_precision = sum([x for x in max_lev_cluster_dict.values()]) / sum(
    [y for x in lev_cluster_dict.values() for y in x.values()])
CEAFm_levenshtein_precision

0.7916741560915986

CEAFm recall

In [37]:
# CEAFm_levenshtein_recall
CEAFm_levenshtein_recall = sum([x for x in max_lev_cluster_dict.values()]) / ents_data.shape[0]
CEAFm_levenshtein_recall

0.7916741560915986

CEAFm Fscore

In [38]:
# CEAFm_levenshtein_f1
CEAFm_levenshtein_f1 = (2 * (CEAFm_levenshtein_recall * CEAFm_levenshtein_precision)) / (
        CEAFm_levenshtein_precision + CEAFm_levenshtein_recall)
CEAFm_levenshtein_f1

0.7916741560915986

### B-CUBED

In [39]:
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in lev_cluster_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(lev_cluster_dict[lev_key][gold_key], 2) /
                                       sum([x for x in lev_cluster_dict[lev_key].values()]))
        except:
            pass
bcubed_precision = bcubed_precision_num/sum([y for x in lev_cluster_dict.values() for y in x.values()])
bcubed_precision

100%|██████████| 5598/5598 [00:36<00:00, 152.36it/s]


0.8723965956665684

In [40]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in lev_cluster_dict.keys():
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(lev_cluster_dict[lev_key][gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num/ents_data.shape[0]
bcubed_recall

100%|██████████| 5598/5598 [00:30<00:00, 186.48it/s]


0.8166030724808002

In [41]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.843578307181706